In [104]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [105]:
# read csv files as df

# Voting record of each county in California
df_ca_poli = pd.read_csv(r'./CA_U.SHouseofRepresentatives.csv', thousands=',')
df_ca_rev = pd.read_csv(r'./TotalRevenueCA2017.csv', thousands=',')

# 
df_ca_rev_clean = df_ca_rev
df_ca_rev_clean["Score"] = df_ca_rev_clean["Percentage"] * 100

# 
ca_key_vals = df_ca_poli.county.unique()
df_ca_poli_clean = pd.DataFrame(columns=["County", "Republican", "Democrat", "Other"])
df_ca_poli_clean["County"] = ca_key_vals

# SNAP
snap_ca = pd.read_csv(r'./SNAPDataCA.csv', thousands=',')
snap_ca.columns = ["County", "SNAP_Percentage", "Total_Pop"]
snap_ca['County'] = snap_ca.County.str.replace('County, CA' , '')
snap_ca['SNAP_Percentage'] = snap_ca['SNAP_Percentage'].str.rstrip('%').astype('float') / 100.0

# Income Bracket
income_brackets_ca = pd.read_csv(r'./IncomeBracketsCA.csv', thousands=',')
income_brackets_ca.columns = ["County", "one_three", "three_eight_four", "Total_Pop"]
income_brackets_ca['County'] = income_brackets_ca.County.str.replace('County, CA' , '')
income_brackets_ca['one_three'] = income_brackets_ca['one_three'].str.rstrip('%').astype('float') / 100.0
income_brackets_ca['three_eight_four'] = income_brackets_ca['three_eight_four'].str.rstrip('%').astype('float') / 100.0

dataframes_to_clean = [df_ca_poli_clean, df_ca_rev, snap_ca, income_brackets_ca]
# for df in dataframes_to_clean:
#     df.columns = df.columns.str.replace(' ', '')
#     df = df.sort_values('County').reset_index(drop=True)
# #     print(df)

def remove_whitespace(df):
    df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    return df

def clean_up_df(df):
    df.columns = df.columns.str.replace(' ', '')
    df = df.sort_values('County').reset_index(drop=True)
    df['County'].replace('', np.nan, inplace=True)
    df.dropna(subset=['County'], inplace=True)
    return df

df_ca_poli_clean = remove_whitespace(df_ca_poli_clean)
df_ca_rev_clean = remove_whitespace(df_ca_rev_clean)
snap_ca = remove_whitespace(snap_ca)
income_brackets_ca = remove_whitespace(income_brackets_ca)

df_ca_poli_clean = df_ca_poli_clean[df_ca_poli_clean.County.str.contains("San Francisco") == False]
df_ca_poli_clean = df_ca_poli_clean[df_ca_poli_clean.County != "Districtwide"]
snap_ca = snap_ca[snap_ca.County.str.contains("San Francisco") == False]
income_brackets_ca = income_brackets_ca[income_brackets_ca.County.str.contains("San Francisco") == False]

df_ca_poli_clean = clean_up_df(df_ca_poli_clean)
df_ca_rev_clean = clean_up_df(df_ca_rev_clean)
snap_ca = clean_up_df(snap_ca)
income_brackets_ca = clean_up_df(income_brackets_ca)
# print(snap_ca)

# San Franisco is not included in the Government Revenue Data Set so we have to remove it. 

assert df_ca_poli_clean.shape[0] == df_ca_rev_clean.shape[0] == snap_ca.shape[0] == income_brackets_ca.shape[0], "Dataframe rows not equal in length"

In [106]:
# GOAL: Process County Level Data to Get Political affiliations of county

# Get Key Values and Set Up New Data Frame

# Calculate percentage
find_affiliation_tot = lambda df, affil, p_value, v_value : df.loc[df[value] == affil][v_value].sum()
for county in ca_key_vals:
    county_df = df_ca_poli.loc[df_ca_poli["county"] == county]
    county_total_votes = county_df["votes"].sum()
    dem_votes = county_df.loc[county_df["party"] == "DEM"]["votes"].sum()
    rep_votes = county_df.loc[county_df["party"] == "REP"]["votes"].sum()
    other_votes = county_df.loc[county_df["party"] == "NPP"]["votes"].sum()

    df_ca_poli_clean.loc[df_ca_poli_clean["County"] == county, ["Republican"]] = (rep_votes/county_total_votes)*100
    df_ca_poli_clean.loc[df_ca_poli_clean["County"] == county, ["Democrat"]] = (dem_votes/county_total_votes)*100
    df_ca_poli_clean.loc[df_ca_poli_clean["County"] == county, ["Other"]] = (other_votes/county_total_votes)*100

df_ca_poli_clean["Score"] = df_ca_poli_clean["Democrat"]
# print(df_ca_poli_clean)

In [107]:
# SNAP Data
snap_ca["Score"] = snap_ca["SNAP_Percentage"] * 100

# Income Brackets
income_brackets_ca["Score"] = (income_brackets_ca["one_three"] + income_brackets_ca["three_eight_four"]) * 100

In [111]:
# Apply metric weights
data_frames = [df_ca_poli_clean, df_ca_rev_clean, snap_ca, income_brackets_ca]
# METRICS
metrics = [.01, 1.5, 1.25, 1.25]
# print(df_ca_rev_clean)
for df, metric in zip(data_frames, metrics):
    df["Score"] = df["Score"].apply(lambda x: x*metric)

# print(df_ca_rev_clean)
print(df_ca_poli_clean)
data_frames = [df_ca_poli_clean, df_ca_rev_clean, snap_ca, income_brackets_ca]
bigdata = pd.concat(data_frames, ignore_index=True)
print(bigdata)
top_ca_counties = processed.sort_values('Score',ascending = False).head(5)
print(top_ca_counties)

             County Republican Democrat      Other     Score
0           Alameda    13.5254  84.9187    1.31708  0.008492
1            Alpine    36.1413  63.8587          0  0.006386
2            Amador     63.405   36.595          0  0.003660
3             Butte    50.2101  48.7744          0  0.004877
4         Calaveras    61.7042  38.2958          0  0.003830
5            Colusa    56.2917  43.7083          0  0.004371
6      Contra Costa    25.3324  69.4382    4.01434  0.006944
7         Del Norte    49.5748  50.4252          0  0.005043
8         El Dorado    58.8502  41.1498          0  0.004115
9            Fresno    53.5882  45.0906   0.787114  0.004509
10            Glenn    63.6746  36.3254          0  0.003633
11         Humboldt    30.3048  69.6952          0  0.006970
12         Imperial     38.868  57.1442    3.98787  0.005714
13             Inyo    55.2075  44.7925          0  0.004479
14             Kern    68.7732  29.5707    1.65611  0.002957
15            Kings    7

/home/ethanc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [114]:
print(top_ca_counties["County"])

76      Los Angeles
37    San Francisco
0           Alameda
20            Marin
43       Santa Cruz
Name: County, dtype: object
